In [2]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [3]:
client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>]

In [4]:
client.create_experiment(name="my-test-experiment")

'2'

In [14]:
from mlflow.entities import ViewType

runs = client.search_runs(
        experiment_ids='1',
        filter_string="metrics.rmse < 6.9",
        run_view_type=ViewType.ACTIVE_ONLY,
        max_results=5,
        order_by=["metrics.rmse ASC"]
)


In [15]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse : {run.data.metrics['rmse']:.4f}")

run id: 53729d50f74e430089dc8b6076dd6c74, rmse : 6.2823
run id: cc034ab1f33c47bcb5ab7a6df80bf6fb, rmse : 6.2823
run id: 8724805422144be9a00d1b47e3da4584, rmse : 6.2897
run id: cdc41b0d727d42b2957fedfa1bc6b848, rmse : 6.2923
run id: 69bebd5df753472fad03a64c4a0b6c51, rmse : 6.2953


In [13]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [16]:
run_id = "69bebd5df753472fad03a64c4a0b6c51"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri,name='nyc_taxi-regressor')

Successfully registered model 'nyc_taxi-regressor'.
2022/05/30 18:49:00 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc_taxi-regressor, version 1
Created version '1' of model 'nyc_taxi-regressor'.


<ModelVersion: creation_timestamp=1653916740482, current_stage='None', description=None, last_updated_timestamp=1653916740482, name='nyc_taxi-regressor', run_id='69bebd5df753472fad03a64c4a0b6c51', run_link=None, source='./mlruns/1/69bebd5df753472fad03a64c4a0b6c51/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [17]:
client.list_registered_models()

[<RegisteredModel: creation_timestamp=1653916738545, description=None, last_updated_timestamp=1653916776089, latest_versions=[<ModelVersion: creation_timestamp=1653916740482, current_stage='Staging', description=None, last_updated_timestamp=1653916776089, name='nyc_taxi-regressor', run_id='69bebd5df753472fad03a64c4a0b6c51', run_link=None, source='./mlruns/1/69bebd5df753472fad03a64c4a0b6c51/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>], name='nyc_taxi-regressor', tags={}>]

In [19]:
model_name = 'nyc_taxi-regressor'
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage : {version.current_stage}")

version: 1, stage : Staging


In [23]:
model_version = 1
new_stage = "Production"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1653916740482, current_stage='Production', description=None, last_updated_timestamp=1653919888758, name='nyc_taxi-regressor', run_id='69bebd5df753472fad03a64c4a0b6c51', run_link=None, source='./mlruns/1/69bebd5df753472fad03a64c4a0b6c51/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [25]:
from datetime import datetime

date = datetime.today().date()

client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned  to {new_stage} on {date}"
)

<ModelVersion: creation_timestamp=1653916740482, current_stage='Production', description='The model version 1 was transitioned  to Production on 2022-05-30', last_updated_timestamp=1653920015424, name='nyc_taxi-regressor', run_id='69bebd5df753472fad03a64c4a0b6c51', run_link=None, source='./mlruns/1/69bebd5df753472fad03a64c4a0b6c51/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>